<a href="https://colab.research.google.com/github/RiqXD/ibm-AI-granite-youtube-analysis/blob/main/Thoriq_Nurul_Mustofa_Capstone_IBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Analisis Data Top 1000 YouTuber Global sebagai Strategi Pembuatan Konten Dengan Model IBM Granite**
By Thoriq Nurul Mustofa


## Import pustaka, data set dan model AI IBM Granite

In [1]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
import pandas as pd

api_token = userdata.get('test_API')
os.environ["REPLICATE_API_TOKEN"] = api_token

model = "ibm-granite/granite-3.3-8b-instruct"
llm = Replicate(model=model,replicate_api_token=api_token,)
df = pd.read_csv("Global YouTube Statistics.csv", encoding="latin1")

## Bersihkan kolom dan baris yang tidak diperlukan

In [2]:
columns_needed = ['Youtuber', 'subscribers', 'video views', 'category', 'Country', 'channel_type']
df = df[columns_needed]
df.columns = ['youtuber', 'subscribers', 'video_views', 'category', 'country', 'channel_type']
df = df.dropna()
df = df[(df['subscribers'] > 0) & (df['video_views'] > 0)]

## Function untuk rekomendasi youtuber berdasarkan input tema dari user

In [3]:
def rekomendasi_konten(df, tema: str, top_n=5):
    tema = tema.lower()
    df_match = df[
        df['category'].str.lower().str.contains(tema) |
        df['channel_type'].str.lower().str.contains(tema)
    ]
    if df_match.empty:
        return None, None

    top_youtuber = df_match.sort_values(by='subscribers', ascending=False).head(top_n)
    kategori_terkait = df_match['category'].value_counts().head(3).index.tolist()
    return top_youtuber, kategori_terkait

# Input from user
tema_user = input("Masukkan tema konten yang ingin kamu buat (contoh: gaming, music, education): ")
top_youtuber, kategori_terkait = rekomendasi_konten(df, tema_user)

if top_youtuber is not None:
    print("\nTop Youtuber sebagai inspirasi:")
    print(top_youtuber[['youtuber', 'subscribers', 'video_views', 'category', 'country']])

    print("\nKategori Terkait:")
    print(", ".join(kategori_terkait))

    summary_text = top_youtuber.to_string(index=False)


    # Prompt to AI model
    prompt = f"""
Saya ingin membuat konten YouTube dengan tema '{tema_user}'. Berikut adalah 5 Youtuber teratas pada tema tersebut:

{summary_text}

Tolong analisis dan berikan rekomendasi strategi pembuatan konten berdasarkan data di atas. Fokus pada:
1. Gaya konten
2. Target audiens
3. Strategi pertumbuhan subscriber
4. Waktu upload atau frekuensi
"""

    # Generate output
    output = llm.invoke(prompt)
    print("\nRekomendasi Strategi :")
    print(output)

else:
    print(f"\nTidak ditemukan Youtuber dengan tema '{tema_user}'. Coba kata kunci lain.")


Masukkan tema konten yang ingin kamu buat (contoh: gaming, music, education): gaming

Top Youtuber sebagai inspirasi:
       youtuber  subscribers   video_views category      country
7     PewDiePie    111000000  2.905804e+10   Gaming        Japan
44  JuegaGerman     48100000  1.463171e+10   Gaming        Chile
50           A4     46300000  2.293663e+10   Gaming         Cuba
52   Fernanfloo     46100000  1.032339e+10   Gaming  El Salvador
81    Mikecrack     39100000  1.611818e+10   Gaming        Spain

Kategori Terkait:
Gaming

Rekomendasi Strategi :
Berdasarkan data yang diberikan, berikut adalah rekomendasi strategi pembuatan konten YouTube berdasarkan tema gaming:

1. **Gaya Konten:**
   - **Variasi**: Youtubers teratas terdapat dalam berbagai negara dan memiliki gaya konten yang berbeda. PewDiePie, misalnya, menggabungkan humor dan narrative ke dalam video game review dan walkthrough. JuegaGerman, di sisi lain, lebih fokus pada tutorial dan gameplay. Fernanfloo dan Mikecrack juga 